In [12]:
import pandas as pd
from datetime import datetime
import tweepy
import numpy as np
import tensorflow as tf
from pythainlp import word_tokenize
# from pythainlp.corpus.common import thai_stopwords
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.backend import clear_session

In [13]:
word = pd.read_csv('dict_file.csv')    
    

In [14]:
# word

In [15]:
Bully_model = tf.keras.models.load_model('cnn&rnn_model.h5')


In [16]:
consumer_key = 'V6WhIk4MThLDHOZffp5ePjhxb'
consumer_secret_key = '0mBgQEhDmlCJYFiEuzv49bwauCKzvBHh7yg2ph5QksMNxHTaOQ'

def OAuth(token,secret_key): # send from user 
    try:
        auth = tweepy.OAuthHandler(consumer_key , consumer_secret_key)
        auth.set_access_token(token,secret_key)
        return auth     
    except Exception as e:
        return none

In [17]:


def tweet(text,from_web_token,from_web_secret_key):
    oauth = OAuth(from_web_token,from_web_secret_key) 
    api = tweepy.API(oauth)
    now = datetime.now()
    time = now.strftime("%b_%d_%Y_%H_%M_%S")
    api.update_status(text+' #'+time)
    return 'ทวีตได้ไม่มีปัญหาครับ'
    

def alert(list_index):
    
    if(len(list_index)==1):
        if(list_index[0]==2):
            alert_text = 'การแบ่งแยก/การกีดกัน (Exclusion)'
        elif(list_index[0]==3):
            alert_text = 'การขมขู่ (Harassment)'
        elif(list_index[0]==4):
            alert_text = 'การแฉ/การเปิดโปง (Revealing)'
        elif(list_index[0]==5):
            alert_text = 'การดูถูก /เหยียดหยาม (Dissing)'      
        elif(list_index[0]==6):
            alert_text = 'การหลอกลวง ,ก่อกวน (Trolling)'
        return alert_text
    else:
        alert_text = 'เป็นการบูลลี่มากกว่าหนึ่งประเภท ได้แก่'
        for x in list_index:
            if(x == 2):
                 alert_text = alert_text +  'การแบ่งแยก/การกีดกัน (Exclusion) และ'
            elif(x == 3):
                    alert_text =alert_text +  'การขมขู่ (Harassment) และ'
            elif(x == 4):
                    alert_text = alert_text + 'การแฉ/การเปิดโปง (Revealing) และ'
            elif(x == 5):
                    alert_text = alert_text + 'การดูถูก /เหยียดหยาม (Dissing) และ'
            elif(x == 6):
                    alert_text = alert_text + 'การหลอกลวง ,ก่อกวน (Trolling)'
        return alert_text


In [18]:
def compare(predict_result):
    
    prepare_list_score = predict_result
    all_score_list = []#ลิสต์ที่จะเอามารองรับตอนที่ทำเป็นเปอร์เซ็นต์
    
    for x in prepare_list_score:
        all_score_list.append(x*100)#ทำให้เป็นเปอร์เซ็นต์จะได้เทียบง่ายๆ
        
    max_score_index = max(all_score_list) #ค่าที่มากที่สุดในลิสต์
    score_for_output = []
    count_index = 0
    
    for score_index in all_score_list:
        
        count_index+=1#เอามานับแทน indexof เพราะมันนับตำแหน่งของเลขซ้ำไม่ได้
        
        if(score_index >= max_score_index * (90/100)): #มากกว่าเท่ากับ 10% จะนับว่าเป็นคลาสนั้นด้วย 
            
#             print(count_index)
            score_for_output.append(count_index)#เอาตำแหน่งของคลาสมาแพ็คเก็บในลิสต์เตรียมส่งออก
    
    return score_for_output #สำหรับแสดงเป็นข้อความแจ้งเตือน ในนี้จะเป็นตำแหน่งของประเภทนั้นๆ

In [19]:
# listmultilabel = [0.5,0.0,0.45,0.0,0.0,0.45]
# compare(listmultilabel)

In [20]:
def getinput(train):
    list_word = []
    proc = word_tokenize(train, engine='newmm')
    list_word.append(proc)
    sequence = list()
    for n in list_word:
        sequenceTemp = list()
        for w in n:
            count=0
            for x in word['Unnamed: 0']:
                count=count+1
                if x==w:
                    sequenceTemp.append(count)
        sequence.append(sequenceTemp)  
    test = np.array(sequence)

    testX = pad_sequences(test, maxlen=105, padding="post")
    return testX

#def getinput(train):
 #   list_word = []
  #  proc = word_tokenize(train, engine='newmm')
   # comment=0
    #comment = [words for words in proc if not words in set(thai_stopwords())]      
    #list_word.append(comment)
    #sequence = list()
    #for n in list_word:
     #   sequenceTemp = list()
      #  for w in n:
       #     count=0
        #    for x in word['Unnamed: 0']:
         #       count=count+1
          #      if x==w:
           #         sequenceTemp.append(count)
        #sequence.append(sequenceTemp)  
   # test = np.array(sequence)
#     testX = pad_sequences(test, maxlen=18, padding="post")
   # testX = pad_sequences(test, maxlen=175, padding="post")
    #return testX

In [21]:
def multiLabel(test):
    ynew =Bully_model.predict(test)
    all = ynew[0][0]+ynew[0][1]+ynew[0][2]+ynew[0][3]+ynew[0][4]+ynew[0][5]
    max = np.amax(ynew[0])
    min = np.amin(ynew[0])
    convert = list()
    for x in ynew[0]:
        result = ((x - min)/100)*100
        convert.append(format(float(result), 'f'))
    return convert

In [22]:
import json
from flask import Flask,request,render_template,url_for,jsonify
app = Flask(__name__) 

@app.route('/') 
def index():
    return render_template('index.php')

@app.route('/main')
def main():
    args = request.args
    global token,secret,username
    token = args['token']
    secret = args['secret']
    username = args['user']
#     pro_picture = args['profile_pic']
    print("token : "+token)
    print("secret : "+secret)
    print("username :"+username)
#     print("picture :"+pro_picture)
    return render_template('main_site.php',show_name=username)
#     return render_template('main_site.php',show_pic=pro_picture)

@app.route('/main_')
def main_():
    return render_template('main_site.php',show_name=username)


@app.route('/main',methods=['GET', 'POST']) 
def output_word():    
    if request.method == 'POST': 
        bully = request.form.get('bully')
        Testinput = getinput(bully)
        output = Bully_model.predict(Testinput)
        real_output = compare(output[0])
        print(real_output)
        if(len(real_output) == 1 and real_output[0] == 1):
            return render_template('alert_succes.html',show=tweet(bully,token,secret))
        else:
            return render_template('alert_error.html',show=alert(real_output))


@app.route('/api', methods=['GET', 'POST'])
def my_json():
    args = request.args
    word = args['word']
    Testinput = getinput(word)
    output = Bully_model.predict(Testinput)
    print(output)
    score = {
        "word" : word,
        "Bully type 1" : json.dumps(str(output[0][0])),
        "Bully type 2" : json.dumps(str(output[0][1])),
        "Bully type 3" : json.dumps(str(output[0][2])),
        "Bully type 4" : json.dumps(str(output[0][3])),
        "Bully type 5" : json.dumps(str(output[0][4])),
        "Bully type 6" : json.dumps(str(output[0][5])),
    }
    return jsonify(score)

#Mark API อย่าลบนะ!
@app.route('/api/input/<string:num>',methods=['GET']) 
def output_word2(num):
        bully =num
        Testinput = getinput(bully)
        output = Bully_model.predict_classes(Testinput)
        multilabel = multiLabel(Testinput)
        #y = tweet(output,bully)
        return jsonify({'Error': 0 , 'value 0' : multilabel[0]
                            , 'value 1' : multilabel[1]
                           , 'value 2' : multilabel[2]
                           , 'value 3' : multilabel[3]
                           , 'value 4' : multilabel[4]
                           , 'value 5' : multilabel[5]})
 

if __name__ == '__main__':
#     app.run(host="0.0.0.0",debug=False, port=5000) 
    app.run(debug=False, port=80) 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:80/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Aug/2020 17:49:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2020 17:49:56] "GET /main?token=247327218-2Y9OmrkjTGBtyKv91zJl7ljwPeEUhkDJIC2lxMnB&secret=Ygy2CPjPaOY3P88UFbmazzNzRU60CviTsJe99VPvlWnBF&user=Ozone HTTP/1.1" 200 -


token : 247327218-2Y9OmrkjTGBtyKv91zJl7ljwPeEUhkDJIC2lxMnB
secret : Ygy2CPjPaOY3P88UFbmazzNzRU60CviTsJe99VPvlWnBF
username :Ozone
[1]


127.0.0.1 - - [12/Aug/2020 17:49:59] "POST /main HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2020 17:50:01] "GET /main_ HTTP/1.1" 200 -


[1]


127.0.0.1 - - [12/Aug/2020 17:50:08] "POST /main HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2020 17:50:09] "GET /main_ HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2020 17:50:19] "POST /main HTTP/1.1" 200 -


[6]


127.0.0.1 - - [12/Aug/2020 17:50:21] "GET /main_ HTTP/1.1" 200 -
